### Cloud Custom VPC with Private Google Access Demo<br>

***

<br>

<img src="../IMG/Screenshot 2023-01-07 222920.png" alt="img" /><br>

**Demo**<br>

Diagram of architecture that we'll be building for this demo<br>
Start by creating a custom VPC<br>
Then going to create 2 subnets: 1 public, 1 private<br>
Cloud Storage buckets with some objects in it<br>
Create some instances to demonstrate the access to the Cloud<br>
As well as the communication between the instances<br>
Finally, will be adding some firewall rules for routing traffics to right places<br>
Also will implement private Google Access, to demonstrate accessibility to files in Cloud<br>
<br>

**How to create a new VPC network:**<br>

Create a new project to have a fresh start<br>
Google Cloud -> VPC Network -> Enable Compute Engine API -> Create VPC network<br>
VPC subnet: custom -> name: public, region: us-east1, IP: 10.0.0.0/24<br>
Add 2nd subnet -> name: private, region: us-east4, IP: 10.0.5.0/24<br>
Enable DNS API and create the VPC<br>

Reminder this can be also done in Cloud shell<br>
You can click the 'gcloud command line' to see how to do it in command line.<br>
<br>

**How to create a new storage bucket:**<br>

Google Cloud -> Storage -> Create bucket<br>
bucket name has to be globally unique<br>
location: region, us-east1<br>

upload some files to the newly created bucket<br>
<br>

**How to create new instances:**<br>

Google Cloud -> VM instances -> create<br>
- public instance:<br>
    - manage label: key: env, value: public<br>
    - region: us-east1, us-east1-b<br>
    - machine type: e2 micro<br>
    - Identity and API access: Access scope: Set access for each API<br>
    Compute Engine: Read/Write, Storage: Read/Write<br>
    - Networking: network tag: public, network interface: custom>subnet /24<br>
<br>

- private instance:<br>
    - manage label: key: env, value: private<br>
    - region: us-east4, us-east4-c<br>
    - machine type: e2 micro<br>
    - Identity and API access: Access scope: Set access for each API<br>
    Storage: Read/Write<br>
    - Networking: network tag: private, network interface: custom>subnet /24<br>
    External IP: None<br>
<br>



**How to create Firewall rules:**<br>

Google Cloud -> VPC Network -> Firewall -> create firewall<br>
- public access/public firewall rule<br>
    - network: custom<br>
    - priority: 1000<br>
    - direction of traffic: ingress<br>
    - action on match: allow<br>
    - target tags: public<br>
    - source IP: 0.0.0.0/0<br>
    - protocols and ports: tcp 22, other protocol: icmp<br>
    - enforcement: enabled<br>
<br>

- private access/private firewall rule<br>
    - network: custom<br>
    - priority: 1000<br>
    - direction of traffic: ingress<br>
    - action on match: allow<br>
    - target tags: private<br>
    - source IP: 10.0.0.0/24, so public instance can access<br>
    - protocols and ports: tcp 22, other protocol: icmp<br>
    - enforcement: enabled<br>
<br>

Can check the new firewall rules in VPC network<br>

Google Cloud -> Compute Engine -> SSH public instance<br>
> gsutil ls gs://FOLDER_NAME<br>

to check whether you can access or not<br>

copy the internal IP of private instance<br>
> ping Internal IP<br>

SSH into the private instance through public instance<br>
> gcloud compute ssh --project PROJECT_NAME --zone us-east4-c private-instance(<- INSTANCE NAME) --internal-ip<br>

prompted with passphrase: (blank)<br>
RSA key is generated<br>

copy the internal IP of public instance<br>
> ping Internal IP<br>
> Ctrl+C to stop<br>
> gsutil ls gs://BUCKET-NAME<br>

You get no response, because external access is needed for cloud storage<br>
but if you want to use this, you can enable Google Private Access<br>
Google Cloud -> VPC network -> Edit private -> turn on Google Access<br>
<br>